<a href="https://colab.research.google.com/github/vaishnavipathak/garbage-classification-cnn/blob/main/cnn_garbage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import os
import wandb
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split


!pip install wandb gdown
!wandb login

!gdown '1cqAJ3b_TIEqHnKQGLGjxCBKS3MUatGa6'


DATA_DIR = "/content/garbage_data/"
!unzip -q Garbage_classification.zip -d {DATA_DIR}

print("Setup complete. Dataset downloaded and unzipped.")

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vaishnavipathak (vaishnavipathak-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
Downloading...
From (original): https://drive.google.com/uc?id=1cqAJ3b_TIEqHnKQGLGjxCBKS3MUatGa6
From (redirected): https://drive.google.com/uc?id=1cqAJ3b_TIEqHnKQGLGjxCBKS3MUatGa6&confirm=t&uuid=4f5bbec0-cd93-4a32-abef-d628b0e6be98
To: /content/Garbage_classification.zip
100% 43.0M/43.0M [00:01<00:00, 32.0MB/s]
Setup complete. Dataset downloaded and unzipped.


In [2]:
# Define Transformations
IMG_SIZE = 224
# We create two pipelines.
# The train transform randomly flips and rotates images for data augmentation.
# The val transform does not, ensuring we validate on unaltered images.
# Both pipelines resize, convert images to PyTorch Tensors, and normalize them.
#  we took Image size as 224*224 and batch size as 32.
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

# Load and Split the Dataset
dataset_folder = os.path.join(DATA_DIR, os.listdir(DATA_DIR)[0])
full_dataset = datasets.ImageFolder(dataset_folder)

train_size = int(0.8 * len(full_dataset)) # 80% training data and 20% data for vatidation.
val_size = len(full_dataset) - train_size
generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

train_dataset.dataset.transform = data_transforms['train']
val_dataset.dataset.transform = data_transforms['val']

#Create DataLoaders
#  Data Loaders is powerful PyTorch tool that groups our dataset into batches and shuffles the training data each epoch to improve learning.
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Verify
print(f"Total images: {len(full_dataset)}")
print(f"Training images: {len(train_dataset)}")
print(f"Validation images: {len(val_dataset)}")
images, labels = next(iter(train_loader))
print(f"\nBatch of images shape: {images.shape}")

Total images: 2527
Training images: 2021
Validation images: 506

Batch of images shape: torch.Size([32, 3, 224, 224])


class SimpleCNN defines our model as a Python class that inherits from PyTorch's base neural network module.

init is the constructor where we define all the layers our network will use.

I have grouped them into two nn.Sequential blocks: self.features for the convolutional part and self.classifier for the final dense layers.


In [3]:
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=6, dense_neurons=256, dropout_rate=0.5):
        """
        Initializes the SimpleCNN model.

        Args:
            num_classes (int): The number of output classes (6 for our dataset).
            dense_neurons (int): The number of neurons in the hidden dense layer.
            dropout_rate (float): The dropout probability.
        """
        super(SimpleCNN, self).__init__()

        # 5 Conv-ReLU-MaxPool blocks as the "feature extractor".
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # defines the dense layer and output layer as the "classifier"
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=32 * 7 * 7, out_features=dense_neurons),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Linear(in_features=dense_neurons, out_features=num_classes)
        )

    def forward(self, x):
        """Defines how data flows through the network."""
        x = self.features(x)
        x = self.classifier(x)
        return x

model = SimpleCNN()
print(model)

SimpleCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): L

In [4]:
import torch.optim as optim

device = torch.device("cuda")
print(f"Using device: {device}")

# instance of the model class we defined earlier
model = SimpleCNN()
# following command moves the model's parameters and computations onto the GPU.
model.to(device)

# Instantiate Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("Model, criterion, and optimizer are ready and configured for GPU.")

Using device: cuda
Model, criterion, and optimizer are ready and configured for GPU.


In [6]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):

    # Loop over the specified number of epochs
    for epoch in range(num_epochs):
        print(f"Starting Epoch {epoch+1}/{num_epochs}")

        # Training Phase
        model.train()
        running_loss = 0.0
        running_corrects = 0

        # Iterate over the training data
        for inputs, labels in train_loader:
            # Move inputs and labels to the GPU
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass, compute predicted outputs
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass, compute gradient of the loss
            loss.backward()

            #Step: update the weights
            optimizer.step()

            # Calculate running statistics
            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)

        # Validation Phase
        model.eval()
        val_loss = 0.0
        val_corrects = 0

        # Disable gradient calculation for validation to save memory and computations.
        with torch.no_grad():
            # Iterate over the validation data
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                _, preds = torch.max(outputs, 1)
                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)

        val_epoch_loss = val_loss / len(val_loader.dataset)
        val_epoch_acc = val_corrects.double() / len(val_loader.dataset)

        print(f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f} | "
              f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}\n")

    print("Training Complete")
    return model

trained_model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10)

Starting Epoch 1/10
Train Loss: 1.2746, Train Acc: 0.4859 | Val Loss: 1.2530, Val Acc: 0.5257

Starting Epoch 2/10
Train Loss: 1.1464, Train Acc: 0.5552 | Val Loss: 1.2408, Val Acc: 0.5138

Starting Epoch 3/10
Train Loss: 1.0817, Train Acc: 0.5898 | Val Loss: 1.0559, Val Acc: 0.6067

Starting Epoch 4/10
Train Loss: 0.9948, Train Acc: 0.6195 | Val Loss: 1.0058, Val Acc: 0.6462

Starting Epoch 5/10
Train Loss: 0.9174, Train Acc: 0.6551 | Val Loss: 1.1207, Val Acc: 0.5968

Starting Epoch 6/10
Train Loss: 0.8556, Train Acc: 0.6843 | Val Loss: 0.9959, Val Acc: 0.6344

Starting Epoch 7/10
Train Loss: 0.8305, Train Acc: 0.6883 | Val Loss: 0.8835, Val Acc: 0.6976

Starting Epoch 8/10
Train Loss: 0.7431, Train Acc: 0.7279 | Val Loss: 0.9158, Val Acc: 0.6719

Starting Epoch 9/10
Train Loss: 0.6968, Train Acc: 0.7288 | Val Loss: 0.8316, Val Acc: 0.7115

Starting Epoch 10/10
Train Loss: 0.6377, Train Acc: 0.7630 | Val Loss: 0.8378, Val Acc: 0.7115

Training Complete
